<h1 align="center"> Phase 2 :  Modeling and Recommendation </h1>

## 1- Preprocessing before graph constraction


*   Convert Type and tail columns to list
*   Check if there is NaN value



In [ ]:
! pip install transformers
! pip install  transformers
! pip install torch_geometric
! pip install torch_geometric
import torch
from torch_geometric.data import Data
import torch
torchversion = torch.__version__
from torch_geometric.data import Data
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
from torch_geometric.utils.convert import from_networkx

In [ ]:
import plotly.graph_objects as go

In [ ]:
import pandas as pd
import torch
import networkx as nx
import pandas as pd
from transformers import BertModel, BertTokenizer
import numpy as np

In [ ]:
df = pd.read_csv("/content/finalDF2.csv")

In [ ]:
df['type'] = df['type'] .apply(lambda x: '[]' if  pd.isna(x) else x)
df['tail']  = df['tail'] .apply(lambda x: '[]' if  pd.isna(x) else x)
df['Synonyms']  = df['Synonyms'] .apply(lambda x: '[]' if  pd.isna(x) else x)

df['Synonyms'] = df['Synonyms'].str.split(',')
def convert_to_valid_string(string):
    return string.replace("'", "\"")

df["type"] = df["type"].apply(convert_to_valid_string)
df["tail"] = df["tail"].apply(convert_to_valid_string)

df["type"] = df["type"].apply(lambda x: eval(x))
df["tail"] = df["tail"].apply(lambda x: eval(x))

In [ ]:
df['Synonyms'] = df['Synonyms'].apply(lambda x: [item.strip("[]") for item in x])

In [ ]:
type(df['Synonyms'][419])

list

In [ ]:
import ast
def str_set_to_list(str_value):
    set_value = ast.literal_eval(str_value)  # Convert the string to a set
    return list(set_value)  # Convert the set to a list

In [ ]:
df['Keyword'] = df['Keyword'].apply(str_set_to_list)

In [ ]:
df=df.rename(columns={'type':'Relations','tail':'Subjects'})
df

,Unnamed: 0.1,Unnamed: 0,Concept,Figure,Keyword,Summary,Relations,Subjects,Synonyms
0,0,0.0,plan risk management,/content/drive/MyDrive/mariem/plan_risk_manage...,"[management process begin project, plan manage...",Plan management is the process of defining how...,"[part of, facet of, use, subclass of, facet of...","[project risk management, project risk managem...","[risk ., risk, management plan, risk, manageme..."
1,1,1.0,project management plan,/content/drive/MyDrive/mariem/identify_risks.p...,"[risk management process, project management c...",in plaing project management all approved subs...,"[use, subclass of, facet of, use, facet of, us...","[project management, subsidiary plan, project,...","[project project, management, component manage..."
2,2,2.0,project document,NaN,"[project document consider, project stakeholde...",project documents can be considered as inputs ...,"[facet of, part of, part of, facet of, part of...","[qualitative analysis, process, project, proce...","[project document project, document]"
3,3,3.0,enterprise environmental factor,NaN,"[environmental factor influence risk, risk man...",Environmental factors can influence the risk m...,[subclass of],[political],"[enterprise, environmental factor enterprise, ..."
4,4,4.0,organizational process asset,NaN,"[organizational process asset, organizational ...",the organizational process assets can influenc...,"[facet of, part of, use, has part, subclass of]","[organization, organization, quantitative anal...","[organizational, process, asset organizational..."
...,...,...,...,...,...,...,...,...,...
415,116,NaN,process illustrate,NaN,"[process illustrate create instance, instance ...",Define 'process illustrate': process illustrat...,"[subclass of, facet of, subclass of]","[project risk, project risk, project risk]",['process illustrate']
416,117,NaN,human activity susceptible bias,NaN,"[human activity susceptible bias, bias human a...",Define 'human activity susceptible bias': huma...,[part of],[four category tool technique],['human activity susceptible bias']
417,118,NaN,restrain force,NaN,"[restrain force currently affect, restrain for...",Define'restrain force': adapt risk identiﬁcati...,"[subclass of, subclass of, subclass of]","[strategic deﬁnition risk response, strategic ...",['restrain force']
418,119,NaN,frequency effort plan plan,NaN,"[plan plan frequency effort, plan frequency ef...",Define 'frequency effort plan plan': frequency...,"[facet of, facet of, facet of]","[risk, risk owner, risk]",['frequency effort plan plan']


In [ ]:
df['Figure'] = df['Figure'].fillna('')

In [ ]:
concatenated_data = {}

# Iterate through the DataFrame
for index, row in df.iterrows():
    concept = row['Concept']
    if concept in concatenated_data:
        concatenated_data[concept]['Figure'].append(row['Figure'])
        concatenated_data[concept]['Summary'].append(row['Summary'])
        concatenated_data[concept]['Keyword'].extend(row['Keyword'])
        concatenated_data[concept]['Relations'].extend(row['Relations'])
        concatenated_data[concept]['Subjects'].extend(row['Subjects'])
        concatenated_data[concept]['Synonyms'].extend(row['Synonyms'])

    else:
        concatenated_data[concept] = {
            'Summary': [row['Summary']],
            'Figure': [row['Figure']],
            'Keyword': row['Keyword'],
            'Relations': row['Relations'],
            'Subjects': row['Subjects'],
            'Synonyms': row['Synonyms'],
        }

# Create a new DataFrame with the concatenated values
result_data = {
    'Concept': [],
    'Summary': [],
    'Figure': [],
    'Keyword': [],
    'Relations': [],
     'Subjects': [],
    'Synonyms': [],
}

for concept, values in concatenated_data.items():
    result_data['Concept'].append(concept)
    result_data['Summary'].append('. '.join(values['Summary']))
    result_data['Figure'].append(', '.join(values['Figure']))
    result_data['Keyword'].append(values['Keyword'])
    result_data['Relations'].append(values['Relations'])
    result_data['Subjects'].append(values['Subjects'])
    result_data['Synonyms'].append(values['Synonyms'])


result_df = pd.DataFrame(result_data)
result_df['Figure'] = result_df['Figure'].fillna('')

In [ ]:
result_df

,Concept,Summary,Figure,Keyword,Relations,Subjects,Synonyms
0,plan risk management,Plan management is the process of defining how...,/content/drive/MyDrive/mariem/plan_risk_manage...,"[management process begin project, plan manage...","[part of, facet of, use, subclass of, facet of...","[project risk management, project risk managem...","[risk ., risk, management plan, risk, manageme..."
1,project management plan,in plaing project management all approved subs...,/content/drive/MyDrive/mariem/identify_risks.p...,"[risk management process, project management c...","[use, subclass of, facet of, use, facet of, us...","[project management, subsidiary plan, project,...","[project project, management, component manage..."
2,project document,project documents can be considered as inputs ...,", , , , , ,","[project document consider, project stakeholde...","[facet of, part of, part of, facet of, part of...","[qualitative analysis, process, project, proce...","[project document project, document, project p..."
3,enterprise environmental factor,Environmental factors can influence the risk m...,", , , ,","[environmental factor influence risk, risk man...","[subclass of, subclass of, subclass of, subcla...","[political, political, political, political, p...","[enterprise, environmental factor enterprise, ..."
4,organizational process asset,the organizational process assets can influenc...,", , , , ,","[organizational process asset, organizational ...","[facet of, part of, use, has part, subclass of...","[organization, organization, quantitative anal...","[organizational, process, asset organizational..."
...,...,...,...,...,...,...,...
366,process illustrate,Define 'process illustrate': process illustrat...,,"[process illustrate create instance, instance ...","[subclass of, facet of, subclass of]","[project risk, project risk, project risk]",['process illustrate']
367,human activity susceptible bias,Define 'human activity susceptible bias': huma...,,"[human activity susceptible bias, bias human a...",[part of],[four category tool technique],['human activity susceptible bias']
368,restrain force,Define'restrain force': adapt risk identiﬁcati...,,"[restrain force currently affect, restrain for...","[subclass of, subclass of, subclass of]","[strategic deﬁnition risk response, strategic ...",['restrain force']
369,frequency effort plan plan,Define 'frequency effort plan plan': frequency...,,"[plan plan frequency effort, plan frequency ef...","[facet of, facet of, facet of]","[risk, risk owner, risk]",['frequency effort plan plan']


# PART_1 : Graph Construction




To construct a graph where each node represents a concept or subject, and edges indicate their relationships, you would:


*  **Identify Nodes** : Define nodes based on key concepts or subjects

*   **Assign Node Features** : Use BERT embeddings to create features for each node. These features include embeddings of keywords, synonyms, and summaries that represent the concept.


 *  **Create Edges** : Establish edges between nodes to show the relationships or connections between different concepts.







In [ ]:
df=result_df

## Feature Embedding using Bert

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Function to convert list to string
def list_to_string(lst):
    return ' '.join(lst)

# Convert lists to strings and concatenate
df['Figure_Str'] = df['Figure'].apply(list_to_string)
df['Keyword_Str'] = df['Keyword'].apply(list_to_string)
df['Synonyms_Str'] = df['Synonyms'].apply(list_to_string)
df['Concatenated_Text'] = df[['Summary', 'Figure_Str', 'Keyword_Str', 'Synonyms_Str']].agg(' '.join, axis=1)

# Get embeddings
df['Embeddings'] = df['Concatenated_Text'].apply(get_bert_embedding)

## Create the graph with NetworkX

In [ ]:
# Create a graph
G = nx.Graph()
array = np.random.rand(1, 768)

# Add nodes for Concepts
for index, row in df.iterrows():
    concept = row['Concept']
    G.add_node(concept,features=row['Embeddings'])
    val=row['Embeddings']
# Add nodes for Subjects
for index, row in df.iterrows():
    Subjects = row['Subjects']
    for sub  in Subjects :
      G.add_node(sub,features= array)

# Add edges (relationships) between Concepts and Subjects
for index, row in df.iterrows():
    concept = row["Concept"]
    relationships = row["Relations"]
    subjects = row["Subjects"]
    for i, rel in enumerate(relationships):
        for subject in subjects:
            G.add_edge(concept, subject, relation=rel)

In [ ]:
from torch_geometric.utils.convert import from_networkx
pyg_graph = from_networkx(G)
pyg_graph

/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/convert.py:260: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  data[key] = torch.tensor(value)


Data(edge_index=[2, 15656], features=[1131, 1, 768], relation=[15656], num_nodes=1131)

**This is a graph representation for 2 nodes**

In [ ]:
sub_df = df[:2]

# Create a graph using NetworkX
G1 = nx.Graph()

# Add nodes (entities and tails) to the graph
nodes = set(sub_df["Concept"].tolist() + [item for sublist in sub_df["Subjects"].tolist() for item in sublist])
G.add_nodes_from(nodes)

# Add edges (relationships) to the graph
for index, row in sub_df.iterrows():
    entity = row["Concept"]
    relationships = row["Relations"]
    tails = row["Subjects"]
    for i, rel in enumerate(relationships):
        tail = tails[i]
        G1.add_edge(entity, tail, relationship=rel)
pos = nx.spring_layout(G1)
fig = go.Figure()

# Add nodes to the figure
for node in G1.nodes:
    x, y = pos[node]
    fig.add_trace(go.Scatter(x=[x], y=[y], mode="markers+text", marker=dict(size=10, color="skyblue"), text=[node]))
# Add edges to the figure
for edge in G1.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_label = edge[2]["relationship"]
    fig.add_trace(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode="lines", line=dict(width=1, color="gray")))
    fig.add_trace(go.Scatter(x=[(x0 + x1) / 2], y=[(y0 + y1) / 2], mode="text", text=[edge_label], textfont=dict(size=8)))

# Configure the figure layout
fig.update_layout(
    title="Interactive Graph Visualization",
    showlegend=False,
    hovermode="closest"
)
fig.show()

## Assign Index Values to Each Nodes

In [ ]:
node_to_index = {node: index for index, node in enumerate(G.nodes)}
G = nx.relabel_nodes(G, node_to_index)

In [ ]:
from torch_geometric.utils.convert import from_networkx
pyg_graph = from_networkx(G)
pyg_graph

Data(edge_index=[2, 15656], features=[1131, 1, 768], relation=[15656], num_nodes=1131)

* The data structure described here seems to be related to a graph in the context of graph neural networks or similar graph-based data processing:

* **edge_index**=[2, 15656]: Indicates the edge connections in the graph, with each column representing an edge. The '2' suggests that these are directed edges (from-to).

* **features**=[1131, 1, 768]: Represents node features in the graph. There are 1131 nodes, each with 768-dimensional feature vectors. The '1' is unclear but might represent a specific feature dimension or a batch size.

* **relation**=[15656]: Suggests that each edge has an associated type or relation, with possibly 15656 unique types or instances.

* **num_nodes**=1131: Indicates the total number of nodes in the graph.

In [ ]:
from gensim.models import Word2Vec
# Extract a list of nodes and edges
nodes = list(G.nodes())
edges = list(G.edges())
print(len(nodes), len(edges))
print()

1131 7832



# PART_2 : Graph Neural Network Architecture

In [ ]:
import torch
torchversion = torch.__version__
from torch_geometric.data import Data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import from_networkx
import torch.nn as nn

# Define a GNN Layer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import from_networkx

class GNNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GNNLayer, self).__init__()
        self.update_fn = nn.Linear(input_dim, output_dim)
        self.aggregate_fn = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)

        # Weight initialization
        nn.init.xavier_uniform_(self.update_fn.weight)
        nn.init.xavier_uniform_(self.aggregate_fn.weight)

    def forward(self, h, adj_matrix):
        messages = torch.matmul(adj_matrix, h)
        aggregated = self.aggregate_fn(messages)
        updated = self.update_fn(h) + aggregated
        h = self.bn(updated)
        return F.relu(h)

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, alpha=0.2, dropout=0.6):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data)

        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj_matrix):
        Wh = torch.mm(h, self.W)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj_matrix > 0, e, zero_vec)

        attention = F.softmax(attention, dim=1)
        attention = self.dropout(attention)

        h_prime = torch.matmul(attention, Wh)

        return F.elu(h_prime)

    def _prepare_attentional_mechanism_input(self, Wh):
        N = Wh.size()[0]
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)
        Wh_repeated_alternating = Wh.repeat(N, 1)
        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        return all_combinations_matrix.view(N, N, 2 * self.out_features)

class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GNN, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GNNLayer(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.layers.append(GNNLayer(hidden_dim, hidden_dim))
        # Replace the last layer with GraphAttentionLayer
        self.layers.append(GraphAttentionLayer(hidden_dim, output_dim))
        self.dropout = nn.Dropout(0.3)

    def forward(self, features, adj_matrix):
        h = features
        for layer in self.layers[:-1]:
            h = layer(h, adj_matrix)
            h = self.dropout(h)
        h = self.layers[-1](h, adj_matrix)
        return h


# Assuming G is your graph object
pyg_graph = from_networkx(G)

# Adjusting features to match the expected shape
features = pyg_graph.features.squeeze(1).float()  # Reshaping to [1131, 768]

# Generate an adjacency matrix from edge_index
edge_index = pyg_graph.edge_index
num_nodes = pyg_graph.num_nodes

# Convert to PyTorch tensor
A = nx.adjacency_matrix(G)
# Convert the sparse matrix to a dense NumPy array
A_dense = A.toarray()
# Convert the NumPy array to a PyTorch tensor of type float
tensor_A = torch.tensor(A_dense, dtype=torch.float32)
adj_matrix=tensor_A

# GNN parameters
input_dim = features.shape[1]  # Assuming 768 as per your features
hidden_dim = 32
output_dim = 8
num_layers = 3


# Creating a GNN model
gnn = GNN(input_dim, hidden_dim, output_dim, num_layers)

# Forward pass through the GNN
embeddings = gnn(features, adj_matrix)

## Trainning

In [ ]:
import random
import torch

def create_edge_set(edge_index):
    return set(tuple(sorted((u.item(), v.item()))) for u, v in edge_index.t())

def sample_negative_edges(num_nodes, edge_set, num_samples):
    neg_edges = []
    while len(neg_edges) < num_samples:
        u, v = random.sample(range(num_nodes), 2)
        if (u, v) not in edge_set and (v, u) not in edge_set:
            neg_edges.append((u, v))
    return torch.tensor(neg_edges).t()

# Assuming edge_index is your original edge index and num_nodes is the number of nodes
edge_set = create_edge_set(edge_index)
num_pos_edges = edge_index.size(1)
num_neg_edges = num_pos_edges

# Shuffle and split positive edges
perm = torch.randperm(num_pos_edges)
train_edge = edge_index[:, perm[:int(0.8 * num_pos_edges)]]
val_test_edge = edge_index[:, perm[int(0.8 * num_pos_edges):]]
val_edge = val_test_edge[:, :val_test_edge.size(1) // 2]
test_edge = val_test_edge[:, val_test_edge.size(1) // 2:]

# Sample negative edges
neg_edge = sample_negative_edges(num_nodes, edge_set, num_neg_edges)
perm = torch.randperm(neg_edge.size(1))
train_neg_edge = neg_edge[:, perm[:int(0.8 * num_neg_edges)]]
val_test_neg_edge = neg_edge[:, perm[int(0.8 * num_neg_edges):]]
val_neg_edge = val_test_neg_edge[:, :val_test_neg_edge.size(1) // 2]
test_neg_edge = val_test_neg_edge[:, val_test_neg_edge.size(1) // 2:]

In [ ]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score, average_precision_score


# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)



def compute_loss(pos_edge_index, neg_edge_index, embeddings):
    # Positive samples loss
    pos_out = torch.sigmoid((embeddings[pos_edge_index[0]] * embeddings[pos_edge_index[1]]).sum(dim=1))
    # Negative samples loss
    neg_out = torch.sigmoid((embeddings[neg_edge_index[0]] * embeddings[neg_edge_index[1]]).sum(dim=1))
    # Combined loss
    return criterion(torch.cat([pos_out, neg_out], dim=0), torch.cat([torch.ones(pos_out.shape[0]), torch.zeros(neg_out.shape[0])], dim=0))

def evaluate(pos_edge_index, neg_edge_index, embeddings):
    with torch.no_grad():
        # Compute outputs for positive and negative edges
        pos_out = torch.sigmoid((embeddings[pos_edge_index[0]] * embeddings[pos_edge_index[1]]).sum(dim=1))
        neg_out = torch.sigmoid((embeddings[neg_edge_index[0]] * embeddings[neg_edge_index[1]]).sum(dim=1))
        predictions = torch.cat([pos_out, neg_out], dim=0)
        labels = torch.cat([torch.ones(pos_out.shape[0]), torch.zeros(neg_out.shape[0])], dim=0)
        # Compute metrics
        roc_score = roc_auc_score(labels.cpu(), predictions.cpu())
        ap_score = average_precision_score(labels.cpu(), predictions.cpu())
        return roc_score, ap_score

# Evaluation

Evaluation using mse

In [ ]:
from sklearn.metrics import mean_squared_error

def evaluate_mse(pos_edge_index, neg_edge_index, embeddings):
    with torch.no_grad():
        # Compute outputs for positive and negative edges
        pos_out = torch.sigmoid((embeddings[pos_edge_index[0]] * embeddings[pos_edge_index[1]]).sum(dim=1))
        neg_out = torch.sigmoid((embeddings[neg_edge_index[0]] * embeddings[neg_edge_index[1]]).sum(dim=1))
        predictions = torch.cat([pos_out, neg_out], dim=0)
        labels = torch.cat([torch.ones(pos_out.shape[0]), torch.zeros(neg_out.shape[0])], dim=0)
        # Compute MSE
        mse_score = mean_squared_error(labels.cpu(), predictions.cpu())
        return mse_score

In [ ]:
# Training Loop
for epoch in range(20):
    optimizer.zero_grad()
    embeddings = gnn(features, adj_matrix)
    loss = compute_loss(train_edge, train_neg_edge, embeddings)
    loss.backward()
    optimizer.step()

    # Validation using MSE
    val_mse = evaluate_mse(val_edge, val_neg_edge, embeddings)
    print(f"Epoch {epoch+1}/{20}, Loss: {loss.item()}, Val MSE: {val_mse}")

# Test Evaluation using MSE
test_mse = evaluate_mse(test_edge, test_neg_edge, embeddings)
print(f"Test MSE: {test_mse}")

Epoch 1/20, Loss: 0.4254634976387024, Val MSE: 0.42107951641082764
Epoch 2/20, Loss: 0.4505898654460907, Val MSE: 0.442587673664093
Epoch 3/20, Loss: 0.5547354221343994, Val MSE: 0.5534278750419617
Epoch 4/20, Loss: 0.4996396005153656, Val MSE: 0.4936542809009552
Epoch 5/20, Loss: 0.5654187202453613, Val MSE: 0.5579053163528442
Epoch 6/20, Loss: 0.43488261103630066, Val MSE: 0.4218057692050934
Epoch 7/20, Loss: 0.5009620785713196, Val MSE: 0.48826122283935547
Epoch 8/20, Loss: 0.5369325876235962, Val MSE: 0.523863673210144
Epoch 9/20, Loss: 0.4038628339767456, Val MSE: 0.3956701159477234
Epoch 10/20, Loss: 0.49535903334617615, Val MSE: 0.4917483329772949
Epoch 11/20, Loss: 0.4367091655731201, Val MSE: 0.4350649118423462
Epoch 12/20, Loss: 0.44635868072509766, Val MSE: 0.4386448860168457
Epoch 13/20, Loss: 0.4811331033706665, Val MSE: 0.46801868081092834
Epoch 14/20, Loss: 0.5486509799957275, Val MSE: 0.5379325747489929
Epoch 15/20, Loss: 0.5072067975997925, Val MSE: 0.5112624168395996


# Part3 : Create recommendation system

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming GNNLayer and GNN are defined as before

class ConceptEmbeddingGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(ConceptEmbeddingGNN, self).__init__()
        self.gnn = GNN(input_dim, hidden_dim, output_dim, num_layers)

    def forward(self, concept_features, adj_matrix):
        concept_embeddings = self.gnn(concept_features, adj_matrix)
        return concept_embeddings

# Define hyperparameters

input_dim = features.shape[1]  # Assuming 768 as per your features
hidden_dim = 32
output_dim = 16
num_layers = 3

In [ ]:
input_dim = features.shape[1]  # Assuming 768 as per your features
hidden_dim = 32
output_dim = 16
num_layers = 3

# Create the model
concept_embedding_model = ConceptEmbeddingGNN(input_dim, hidden_dim, output_dim, num_layers)

In [ ]:
# Create the model
concept_embedding = concept_embedding_model(features, adj_matrix)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
class QueryEmbeddingTransform(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QueryEmbeddingTransform, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, x):
        return F.relu(self.linear(x))

In [ ]:
transform_model = QueryEmbeddingTransform(768, 8)  # From BERT size to your concept embedding size

In [ ]:
def embed_query(query):
    # Embed the query using BERT as before
    inputs = tokenizer(query, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    query_embedding = outputs.last_hidden_state.mean(dim=1)

    # Transform the query embedding to match the concept embedding size
    transformed_embedding = transform_model(query_embedding)
    return transformed_embedding

In [ ]:
import torch

# Sample test queries
test_queries = ["risk assessment techniques", "financial risk management", "data security risks","plan risk management tools","what is project management "]

# Function to calculate similarity
def get_similarity(query_embedding, concept_embeddings):
    # Cosine similarity - ensure embeddings are normalized if necessary
    similarity = torch.nn.functional.cosine_similarity(query_embedding, concept_embeddings, dim=1)
    return similarity

# Generate test concept embeddings
# Assuming 'concept_embedding_model', 'features', and 'adj_matrix' are defined and initialized
concept_embeddings = concept_embedding_model(features, adj_matrix)

# Testing with each query
for query in test_queries:
    query_embedding = embed_query(query)  # Ensure this function returns a normalized embedding if necessary
    similarity_scores = get_similarity(query_embedding.unsqueeze(0), embeddings)

    # Get top-k similar concepts
    top_k_values, top_k_indices = torch.topk(similarity_scores, k=3)
    print(f"Top concepts for query '{query}': {top_k_indices.tolist()}")
    elements = [top_k_indices[0, i].item() for i in range(top_k_indices.size(1))]
    for key,value in node_to_index.items() :
       for i in elements :
        if i==value :
          print(key)
    print("**************************************************************")
    print('\n')

Top concepts for query 'risk assessment techniques': [[2, 5, 1]]
project management plan
project document
expert judgment
**************************************************************


Top concepts for query 'financial risk management': [[1, 7, 3]]
project management plan
enterprise environmental factor
meeting
**************************************************************


Top concepts for query 'data security risks': [[7, 4, 5]]
organizational process asset
expert judgment
meeting
**************************************************************


Top concepts for query 'plan risk management tools': [[2, 5, 1]]
project management plan
project document
expert judgment
**************************************************************


Top concepts for query 'what is project management ': [[7, 4, 5]]
organizational process asset
expert judgment
meeting
**************************************************************




In [ ]:
concept_embeddings

In [ ]:
import tensorflow as tf

# Replace "your_model_file.h5" with the actual file name of your exported model
model_file_path = "/content/gnn.h5"

In [ ]:
model = ConceptEmbeddingGNN(input_dim, hidden_dim, output_dim, num_layers)
model.load_state_dict(torch.load('gnn.h5'))


In [ ]:
concept_embedding

In [ ]:
filename = 'gnn_model.h5'
torch.save(concept_embeddings, filename)